In [1]:
from tkinter import *
from tkinter.ttk import *
from tkinter.scrolledtext import ScrolledText
from cmp.pycompiler import *
from LL1Test import *
from SLR1ParserTest import *
from LR1Test import *
from LALRTest import *
from GrammarOperations import *

A
[B, a]
B
[b]
b
a


In [2]:
def JumpWhiteSpaces(text, index):
    while index < len(text) and (text[index] == ' ' or text[index] == '\n'):
            index += 1
            
    if index == len(text):
        return None
    
    return index        

def ConsumeToWhiteSpace(text, index):
    result = ""
    while index < len(text) and text[index] != ' ' and text[index] != '\n' and text[index] != ';' and text[index] != '|' and text[index] != '-' and text[index] != '>':
        result += text[index]
        index += 1
    
    if len(result) == 0:
        if text[index] == '|':
            result = '|'
            index += 1
            
        elif text[index] == ';':
            result = ';'
            index += 1
            
        else:    
            return None, index
    
    return result, index

def ConsumeArrow(text, index):
    index = JumpWhiteSpaces(text, index)
    if index is None:
        return len(text), False
    
    if index + 2 >= len(text):
        return index, False
     
    if text[index] != '-' or text[index + 1] != '-' or text[index + 2] != '>':
        return index, False
    
    index += 3
    
    return index ,True    

def FormProduction(text, index, G, symbols):
    
    right = [[]]
    i = 0
    while True:
       
        index = JumpWhiteSpaces(text, index)
        if index is None:
            return None, len(text)
        
        string, index = ConsumeToWhiteSpace(text, index)
        
        
        if string is None:
            return None, index
        
        if string == ";":
            if len(right[i]) == 0:
                return None, index
            
            return right, index
        
        if string == '|':
            i += 1
            right.append([])
            continue
        
        
        try:
            right[i].append(symbols[string])
            
        except KeyError:
            if string[0] >= 'A' and string[0] <= 'Z':
                s = G.NonTerminal(string)
                right[i].append(s)
                symbols[string] = s
            else:
                s = G.Terminal(string)
                right[i].append(s)
                symbols[string] = s
                
            

def GetGrammar(text, symbols):
    
    if len(text) == 1:
        lbl.insert('1.0',"Escribe la gramatica")
        return None
    
    start = True
    index = 0
    

    G = Grammar()
    symbols['e'] = G.Epsilon
    symbols['\n'] = G.EOF
    while index < len(text):
        #ignorar espacios en blanco
        index = JumpWhiteSpaces(text, index)
        if index is None:
            return G
        
        if text[index] < 'A' or text[index] > 'Z':
            lbl.insert('1.0',"Se esperaba un no Terminal " + text[index])
            return None
        
        current, index = ConsumeToWhiteSpace(text, index)
        
        if current is None:
            return G
        
        if start:
            left = symbols[current] = G.NonTerminal(current, True)
            start = False
            
        else:
            try:
                left = symbols[current]
            except KeyError:
                left = symbols[current] = G.NonTerminal(current)
        
        
        index, Ok = ConsumeArrow(text, index)
        if not Ok:
            lbl.insert('1.0', "Se esperaba --> " + text[index - 1])
            return None
           
        right, index = FormProduction(text, index, G, symbols) 
        
        if right is None:
            lbl.insert('1.0', "Se esperaba una produccion " + text[index - 1])
            return None
    
        for productions in right:
            sentence = productions[0]
            for i in range(1, len(productions)):
                sentence = sentence + productions[i] + G.Epsilon
            
            left %= sentence
    
    return G
        
def ToTerminals(cadena, symbols):
    string = []
    for char in cadena:
        string.append(symbols[char])
        
    return string    

def clicked():
    symbols = {}
    grammar_type = combo.get()
    lbl.delete("1.0", END)
       
    G = GetGrammar(st.get("1.0", END), symbols)
    def parsea():
        cadena = entry.get()
        string = ToTerminals(cadena, symbols)
        string.append(G.EOF)
        if grammar_type == 'll1':
            derivation_tree = Derivation_Tree(string, parser)
            
        else:
            derivation_tree = Derivation_Tree_ShiftReduce(string, parser)
        l.configure(text = string)
        
    window2 = Tk()
    window2.title("Parsea la cadena")
    window2.geometry('600x500')
    btn2 = Button(window2, text = "Click", command = parsea)
    btn2.grid(column = 1, row = 0)

    entry = Entry(window2)
    entry.grid(column = 0, row = 1)
    l = Label(window2)
    l.grid(column = 0, row = 2)
    
   
    if not G is None:
        text = ""
        
        if grammar_type == 'll1':
            text1, automata_regular = Execute(G)
            text += text1
            text1, parser = Execute_LL1(G)
            text += text1

        elif grammar_type == 'slr':
            text1, automata, parser = Execute_SLR1(G)
            text += text1

        elif grammar_type == 'lr1':
            text1, automata, parser = Execute_LR1(G)
            text += text1

        elif grammar_type == 'lalr':
            text1, automata, parser = Execute_LALR(G)
            text += text1
        
        if not grammar_type == 'll1':
            text1, automata_regular = Execute(G)
            text += text1
            
        
        lbl.insert('1.0', text)
        
    print("None")    

In [3]:
window = Tk()
window.title("Gestor de Gramatica")
window.geometry('600x500')

combo = Combobox(window)
combo['values'] = ('ll1', 'slr','lr0', 'lr1', 'lalr', 'regular')
combo.current(0)
combo.grid(column = 0, row = 0)

btn = Button(window, text = "Click", command = clicked)
btn.grid(column = 1, row = 0)

st = ScrolledText(window, width = 35, height = 30)
st.grid(column = 0, row = 1)


lbl = ScrolledText(window, width = 111, height = 30)
lbl.grid(column = 1, row = 1)


window.mainloop()



None
